In [20]:
import uuid
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq
from langgraph.graph import START, END, StateGraph
from langchain_core.messages import HumanMessage, BaseMessage, SystemMessage, AIMessage
from langgraph.graph.message import add_messages
from langgraph.store.base import BaseStore
from langchain_core.runnables import RunnableConfig
from dotenv import load_dotenv
from typing import TypedDict, Annotated, List
from langgraph.store.memory import InMemoryStore
from CONFIG import GROQ_MODEL, TEMPERATURE

In [14]:
load_dotenv()
llm = ChatGroq(model=GROQ_MODEL, temperature=TEMPERATURE)

In [17]:
class pydantic_class(BaseModel):
    should_store: bool = Field(description="whether to store any memory")
    list_of_memory : List[str] = Field(default_factory=List, description="atomic user memory to store")

structured_llm = llm.with_structured_output(pydantic_class)

In [9]:
class state_class(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [ ]:
def stored_LTM_memory(state: state_class, config: RunnableConfig, store: BaseStore):
    config = config['configurable']['user_id']
    namespace = ('user', config, 'details')
    last_msg = state['messages'][-1].content

    decision: pydantic_class = structured_llm.invoke(
        [
            SystemMessage(
                content=(
				"Extract LONG-TERM memories from the user's message.\n"
				"Only store stable, user-specific info (identity, preferences, ongoing projects).\n"
				"Do NOT store transient info.\n"
				"Return should_write=false if nothing is worth storing.\n"
				"Each memory should be a short atomic sentence."
                )
		)
	  ]
    )